In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F0'
SS_VER = 'SS_VER_2_1'
%matplotlib widget


In [2]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        print("SS_VER_2_1")
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        print("SimpleSerial")
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")

SS_VER_2_1
INFO: Found ChipWhisperer😍


In [3]:
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
    print("cw.programmers.STM32FProgrammer")
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif "SAM4S" in PLATFORM or PLATFORM == "CWHUSKY":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None

cw.programmers.STM32FProgrammer


In [4]:
prog = cw.programmers.STM32FProgrammer
import time
time.sleep(0.05)
scope.default_setup()

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 266356472                 to 287090411                
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 96000000                  to 29538459                 
scope.clock.adc_rate                     changed from 96000000.0                to 29538459.0               
scope.clock.clkgen_

In [5]:
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        print("CW308_SAM4S clause")
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:
        # print("else clause")
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

In [16]:
%%bash -s "$PLATFORM" "$SS_VER"

cd mcu/basic-passwdcheck
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CW308_STM32F0 
.
Welcome to another exciting ChipWhisperer target build!!
.
.
.
Compiling:
.
Compiling:
Compiling:
-en     basic-passwdcheck.c ...
Compiling:
-en     .././simpleserial/simpleserial.c ...
-en     .././hal/hal.c ...
.
-en     .././hal//stm32f0/stm32f0_hal.c ...
Compiling:
.
-en     .././hal//stm32f0/stm32f0_hal_lowlevel.c ...
Assembling: .././hal//stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F0/stm32f0_startup.lst -I.././simpleserial/ -I.././hal/ -I.././hal/ -I.././hal//stm32f0 -I.././hal//stm32f0/CMSIS -I

In [17]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, "mcu/basic-passwdcheck/basic-passwdcheck-CW308_STM32F0.hex")


Detected unknown STM32F ID: 0x440
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4843 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4843 bytes


In [18]:
# target.baud = 38400  # or 115200

# def read_until_prompt(prompt="Please enter password to continue: ", timeout=2.0):
#     """Drain UART until the password prompt appears. Works for str or bytes returns."""
#     t0 = time.time()
#     buf = ""  # use string buffer consistently
#     while time.time() - t0 < timeout:
#         chunk = target.read(timeout=0.1)
#         if not chunk:
#             continue
#         if isinstance(chunk, bytes):
#             chunk = chunk.decode(errors="ignore")
#         buf += chunk
#         if prompt in buf:
#             return buf
#     return buf  # whatever we collected
def cap_pass_trace(pass_guess):
    reset_target(scope)
    # _banner = read_until_prompt()
    # print(_banner)
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

In [19]:
scope.adc.samples = 3000

In [27]:

trace_test = cap_pass_trace("h\n")

#Basic sanity check
assert(len(trace_test) == 3000)
print("✔️ OK to continue!")

✔️ OK to continue!


In [46]:
# Try the baud used by your lab firmware; many 2_1B builds are 38400
target.baud = 38400  # or 115200

scope.trigger.triggers = "tio1"
scope.adc.samples = 5000
scope.adc.offset = 0
scope.adc.decimate = 1
scope.adc.timeout = 2.0
scope.io.target_pwr = True

def reset_target():
    scope.io.nrst = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.15)

def read_until_prompt(prompt="Please enter password to continue: ", timeout=2.0):
    """Drain UART until the password prompt appears. Works for str or bytes returns."""
    t0 = time.time()
    buf = ""  # use string buffer consistently
    while time.time() - t0 < timeout:
        chunk = target.read(timeout=0.1)
        if not chunk:
            continue
        if isinstance(chunk, bytes):
            chunk = chunk.decode(errors="ignore")
        buf += chunk
        if prompt in buf:
            return buf
    return buf  # whatever we collected

def capture_one(password_line: str):
    """Reset, wait for prompt, arm, send password (must end with \\n), capture."""
    reset_target()

    # Option A (robust): wait until we actually see the prompt
    _banner = read_until_prompt()
    print(_banner)
    # Option B (quick): just flush instead of reading the banner
    # target.flush()

    scope.arm()
    target.write(password_line)  # must include '\n' to exit my_read()

    if scope.capture():
        print("Timeout: no trigger seen (check baud, trigger pin, newline).")
        trace = None
    else:
        trace = scope.get_last_trace()
        print("Captured trace length:", len(trace))

    # Read app response (FAIL / Access granted)
    resp = target.read(timeout=0.5)
    if isinstance(resp, bytes):
        resp = resp.decode(errors="ignore")
    print("Target said:", (resp or "").strip())

    return trace, resp

# Example:
# trace, resp = capture_one("guessme\n")   # wrong
trace, resp = capture_one("h0px3\n")   # correct

    ! *****Safe-o-matic 3000 Booting...
Decrypting database..[DONE]


Please enter password to continue: 
Captured trace length: 5000
Target said: Access gr


In [39]:
import chipwhisperer as cw

# --- Capture a couple of traces ---
trace1 = cap_pass_trace("h\n")
trace2 = cap_pass_trace("a\n")


# --- Plot them with ChipWhisperer helper ---
cw.plot(trace1, label="h")*cw.plot(trace2, label="a")



:Overlay
   .Curve.H :Curve   [x]   (y)
   .Curve.A :Curve   [x]   (y)

In [30]:
from cwtraces import sca101_lab_data

cap_pass_trace = sca101_lab_data["lab2_1"]["cap_pass_trace"]

trace_test = cap_pass_trace("h\n")

print("Trace length:", len(trace_test))
cw.plot(cap_pass_trace("h\n")) * cw.plot(cap_pass_trace("0\n"))


Trace length: 3000


:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [14]:
scope

cwlite Device
sn         = 50203220343043543230323238303035
fw_version = 
    major = 0
    minor = 65
    debug = 0
gain = 
    mode = high
    gain = 30
    db   = 24.8359375
adc = 
    state          = True
    basic_mode     = rising_edge
    timeout        = 2
    offset         = 0
    presamples     = 0
    samples        = 3000
    decimate       = 1
    trig_count     = 1771909994
    fifo_fill_mode = normal
clock = 
    adc_src       = clkgen_x4
    adc_phase     = 0
    adc_freq      = 29538459
    adc_rate      = 29538459.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1         = serial_rx
    tio2         = serial_tx
    tio3         = high_z
    tio4         = high_z
    pdid         = high_z
    pdic         = high_z
